In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal, stats
from scipy.interpolate import interp1d
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
import os
import warnings
import joblib
warnings.filterwarnings('ignore')

2025-10-27 00:27:31.151305: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-27 00:27:31.207755: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-27 00:27:32.531779: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
class Config:
    # Paths
    CURRENT_TEMP_PATH = '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/0NM_current_temp_csv'
    VIBRATION_PATH = '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/0nm_vibration_csv'
    
    # Sampling parameters
    ORIGINAL_FREQ = 25600  # Hz
    TARGET_FREQ = 1000     # Hz
    NORMAL_DURATION = 120  # seconds
    FAULTY_DURATION = 60   # seconds
    
    # Window parameters
    WINDOW_SIZE = 2000     # 2 seconds at 1kHz
    OVERLAP = 0.5          # 50% overlap
    
    # Fault severity mapping (in mm)
    SEVERITY_MAP = {
        'Normal': 0.0,
        'BPFI_03': 0.3,
        'BPFI_10': 1.0,
        'BPFI_30': 3.0
    }
    
    # Model parameters
    BATCH_SIZE = 32
    EPOCHS = 100
    LEARNING_RATE = 0.001
    
    # Synthetic data parameters
    NUM_SYNTHETIC_PATHS = 50
    AUGMENTATION_FACTOR = 3

config = Config()

In [3]:
def load_and_prepare_test_data(temp_filepath, vib_filepath, scaler_X, sequence_length=10):
    """
    Load original CSV files and prepare them for model testing
    
    Args:
        temp_filepath: Path to current_temp CSV file
        vib_filepath: Path to vibration CSV file
        scaler_X: Fitted feature scaler from training
        sequence_length: Number of timesteps for LSTM
    
    Returns:
        X_test_scaled: Prepared sequences ready for model prediction
        feature_df: Extracted features (for inspection)
    """
    print(f"\n{'='*70}")
    print("LOADING AND PREPARING TEST DATA")
    print(f"{'='*70}")
    
    # Step 1: Load CSV files
    print(f"\n[1/5] Loading CSV files...")
    df_temp = pd.read_csv(temp_filepath)
    df_vib = pd.read_csv(vib_filepath)
    print(f"✓ Temperature data: {df_temp.shape}")
    print(f"✓ Vibration data: {df_vib.shape}")
    
    # Step 2: Preprocess (resample from 25.6kHz to 1kHz)
    print(f"\n[2/5] Resampling from {config.ORIGINAL_FREQ}Hz to {config.TARGET_FREQ}Hz...")
    df_temp_resampled = preprocess_dataframe(df_temp, 'current_temp')
    df_vib_resampled = preprocess_dataframe(df_vib, 'vibration')
    print(f"✓ Temperature resampled: {df_temp_resampled.shape}")
    print(f"✓ Vibration resampled: {df_vib_resampled.shape}")
    
    # Step 3: Extract features from windows
    print(f"\n[3/5] Extracting features from {config.WINDOW_SIZE} sample windows...")
    feature_df = create_windowed_features(
        df_temp_resampled, 
        df_vib_resampled, 
        config.WINDOW_SIZE, 
        config.OVERLAP
    )
    print(f"✓ Extracted features: {feature_df.shape}")
    print(f"  Number of windows: {feature_df.shape[0]}")
    print(f"  Number of features per window: {feature_df.shape[1]}")
    
    # Step 4: Create sequences for LSTM
    print(f"\n[4/5] Creating sequences of length {sequence_length}...")
    X = feature_df.values
    
    # Create sequences (sliding window approach)
    X_sequences = []
    for i in range(len(X) - sequence_length + 1):
        X_sequences.append(X[i:i+sequence_length])
    
    X_sequences = np.array(X_sequences)
    print(f"✓ Created {X_sequences.shape[0]} sequences")
    print(f"  Sequence shape: {X_sequences.shape}")
    
    # Step 5: Scale features using the same scaler from training
    print(f"\n[5/5] Scaling features...")
    X_reshaped = X_sequences.reshape(-1, X_sequences.shape[-1])
    X_scaled = scaler_X.transform(X_reshaped)
    X_test_scaled = X_scaled.reshape(X_sequences.shape)
    print(f"✓ Features scaled")
    
    print(f"\n{'='*70}")
    print("DATA PREPARATION COMPLETE!")
    print(f"{'='*70}")
    print(f"Ready for prediction: {X_test_scaled.shape}")
    
    return X_test_scaled, feature_df

In [4]:
def test_model_on_original_data(temp_filepath, vib_filepath, 
                                  model_path='predictive_maintenance_model.h5',
                                  scaler_X_path='scaler_X.pkl',
                                  scaler_y_path='scaler_y.pkl',
                                  sequence_length=10):
    """
    Complete pipeline to test trained model on original data
    
    Args:
        temp_filepath: Path to temperature CSV
        vib_filepath: Path to vibration CSV
        model_path: Path to saved model
        scaler_X_path: Path to feature scaler
        scaler_y_path: Path to target scaler
        sequence_length: LSTM sequence length
    
    Returns:
        predictions_df: DataFrame with all predictions and analysis
    """
    print(f"\n{'='*70}")
    print("TESTING MODEL ON ORIGINAL DATA")
    print(f"{'='*70}")
    
    # Load model and scalers
    print("\n[1/3] Loading trained model and scalers...")
    try:
        model = keras.models.load_model(model_path,compile=False)
        scaler_X = joblib.load(scaler_X_path)
        scaler_y = joblib.load(scaler_y_path)
        print("✓ Model and scalers loaded successfully")
    except Exception as e:
        print(f"✗ Error loading model/scalers: {e}")
        return None
    
    # Prepare test data
    print("\n[2/3] Preparing test data...")
    X_test_scaled, feature_df = load_and_prepare_test_data(
        temp_filepath, 
        vib_filepath, 
        scaler_X, 
        sequence_length
    )
    
    # Make predictions
    print("\n[3/3] Making predictions...")
    y_pred_scaled = model.predict(X_test_scaled, verbose=1)
    y_pred = scaler_y.inverse_transform(y_pred_scaled).flatten()
    
    print(f"✓ Generated {len(y_pred)} predictions")
    
    # Create results DataFrame
    results = {
        'sequence_index': np.arange(len(y_pred)),
        'predicted_RUL_minutes': y_pred,
        'predicted_RUL_hours': y_pred / 60,
        'health_state': ['Healthy' if r > 60 else 'Warning' if r > 20 else 'Critical' for r in y_pred],
        'maintenance_urgency': ['Low' if r > 60 else 'Medium' if r > 20 else 'High' for r in y_pred]
    }
    
    predictions_df = pd.DataFrame(results)
    
    # Add statistics
    print(f"\n{'='*70}")
    print("PREDICTION SUMMARY")
    print(f"{'='*70}")
    print(f"Total sequences analyzed: {len(y_pred)}")
    print(f"\nRUL Statistics:")
    print(f"  Mean RUL: {y_pred.mean():.2f} minutes ({y_pred.mean()/60:.2f} hours)")
    print(f"  Min RUL: {y_pred.min():.2f} minutes")
    print(f"  Max RUL: {y_pred.max():.2f} minutes")
    print(f"  Std Dev: {y_pred.std():.2f} minutes")
    
    print(f"\nHealth State Distribution:")
    print(predictions_df['health_state'].value_counts())
    
    # Identify critical sequences
    critical_sequences = predictions_df[predictions_df['health_state'] == 'Critical']
    if len(critical_sequences) > 0:
        print(f"\n⚠️  WARNING: {len(critical_sequences)} CRITICAL sequences detected!")
        print(f"   Minimum RUL: {critical_sequences['predicted_RUL_minutes'].min():.2f} minutes")
        print(f"   Action: IMMEDIATE MAINTENANCE REQUIRED!")
    else:
        print(f"\n✓ No critical conditions detected")
    
    # Save results
    output_filename = 'test_predictions.csv'
    predictions_df.to_csv(output_filename, index=False)
    print(f"\n✓ Predictions saved to '{output_filename}'")
    
    return predictions_df, y_pred, feature_df

In [5]:
def visualize_test_predictions(predictions_df, y_pred):
    """
    Visualize predictions on test data
    
    Args:
        predictions_df: DataFrame with prediction results
        y_pred: Array of predicted RUL values
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. RUL over time
    ax1 = axes[0, 0]
    ax1.plot(predictions_df['sequence_index'], predictions_df['predicted_RUL_minutes'], 
             linewidth=2, color='blue', alpha=0.7)
    ax1.axhline(60, color='green', linestyle='--', linewidth=2, label='Healthy Threshold')
    ax1.axhline(20, color='red', linestyle='--', linewidth=2, label='Critical Threshold')
    ax1.fill_between(predictions_df['sequence_index'], 0, 20, alpha=0.2, color='red')
    ax1.fill_between(predictions_df['sequence_index'], 20, 60, alpha=0.2, color='orange')
    ax1.fill_between(predictions_df['sequence_index'], 60, predictions_df['predicted_RUL_minutes'].max(), 
                     alpha=0.2, color='green')
    ax1.set_xlabel('Sequence Index', fontsize=12)
    ax1.set_ylabel('Predicted RUL (minutes)', fontsize=12)
    ax1.set_title('RUL Progression Over Time', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Health state distribution
    ax2 = axes[0, 1]
    health_counts = predictions_df['health_state'].value_counts()
    colors = ['#22c55e' if state == 'Healthy' else '#f59e0b' if state == 'Warning' else '#ef4444' 
              for state in health_counts.index]
    bars = ax2.bar(health_counts.index, health_counts.values, color=colors, edgecolor='black')
    ax2.set_ylabel('Count', fontsize=12)
    ax2.set_title('Health State Distribution', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    
    for bar, count in zip(bars, health_counts.values):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{count}\n({count/len(predictions_df)*100:.1f}%)',
                ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # 3. RUL histogram
    ax3 = axes[1, 0]
    ax3.hist(y_pred, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    ax3.axvline(y_pred.mean(), color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {y_pred.mean():.1f} min')
    ax3.axvline(60, color='green', linestyle=':', linewidth=2, alpha=0.7)
    ax3.axvline(20, color='red', linestyle=':', linewidth=2, alpha=0.7)
    ax3.set_xlabel('Predicted RUL (minutes)', fontsize=12)
    ax3.set_ylabel('Frequency', fontsize=12)
    ax3.set_title('RUL Distribution', fontsize=14, fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3, axis='y')
    
    # 4. Moving average
    ax4 = axes[1, 1]
    window = min(10, len(y_pred) // 10)
    if window > 0:
        moving_avg = pd.Series(y_pred).rolling(window=window, min_periods=1).mean()
        ax4.plot(predictions_df['sequence_index'], y_pred, 
                alpha=0.3, color='blue', label='Raw Predictions')
        ax4.plot(predictions_df['sequence_index'], moving_avg, 
                linewidth=2, color='darkblue', label=f'Moving Avg (window={window})')
        ax4.axhline(20, color='red', linestyle='--', linewidth=2, alpha=0.5)
        ax4.set_xlabel('Sequence Index', fontsize=12)
        ax4.set_ylabel('RUL (minutes)', fontsize=12)
        ax4.set_title('RUL with Moving Average', fontsize=14, fontweight='bold')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('test_predictions_visualization.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("✓ Visualization saved as 'test_predictions_visualization.png'")



def preprocess_dataframe(df, data_type='current_temp'):
    """Preprocess dataframe: resample and clean"""
    df_processed = df.copy()
    
    # Remove timestamp column if exists
    if 'timestamp' in df_processed.columns:
        df_processed = df_processed.drop('timestamp', axis=1)
    
    # Resample all columns
    resampled_data = {}
    for col in df_processed.columns:
        resampled_data[col] = resample_signal(
            df_processed[col].values,
            config.ORIGINAL_FREQ,
            config.TARGET_FREQ
        )
    
    df_resampled = pd.DataFrame(resampled_data)
    
    # Handle NaN and infinite values
    df_resampled = df_resampled.replace([np.inf, -np.inf], np.nan)
    df_resampled = df_resampled.fillna(method='ffill').fillna(method='bfill')
    
    return df_resampled

def resample_signal(signal_data, original_freq, target_freq):
    """Resample signal from original frequency to target frequency"""
    num_samples = len(signal_data)
    duration = num_samples / original_freq
    target_samples = int(duration * target_freq)
    
    # Use scipy's resample for high-quality resampling
    resampled = signal.resample(signal_data, target_samples)
    return resampled

def create_windowed_features(df_temp, df_vib, window_size, overlap):
    """Create windowed features from sensor data"""
    step_size = int(window_size * (1 - overlap))
    num_windows = (len(df_temp) - window_size) // step_size + 1
    
    features_list = []
    
    for i in range(num_windows):
        start_idx = i * step_size
        end_idx = start_idx + window_size
        
        if end_idx > len(df_temp):
            break
        
        window_features = {}
        
        # Temperature features
        for col in df_temp.columns:
            temp_window = df_temp[col].values[start_idx:end_idx]
            time_feats = extract_time_domain_features(temp_window)
            for feat_name, feat_val in time_feats.items():
                window_features[f'temp_{col}_{feat_name}'] = feat_val
        
        # Vibration features
        for col in df_vib.columns:
            vib_window = df_vib[col].values[start_idx:end_idx]
            time_feats = extract_time_domain_features(vib_window)
            freq_feats = extract_frequency_domain_features(vib_window)
            for feat_name, feat_val in {**time_feats, **freq_feats}.items():
                window_features[f'vib_{col}_{feat_name}'] = feat_val
        
        features_list.append(window_features)
    
    return pd.DataFrame(features_list)


def extract_time_domain_features(window):
    """Extract time-domain features from a signal window"""
    features = {}
    features['mean'] = np.mean(window)
    features['std'] = np.std(window)
    features['rms'] = np.sqrt(np.mean(window**2))
    features['peak'] = np.max(np.abs(window))
    features['peak_to_peak'] = np.ptp(window)
    features['crest_factor'] = features['peak'] / (features['rms'] + 1e-8)
    features['kurtosis'] = stats.kurtosis(window)
    features['skewness'] = stats.skew(window)
    return features

def extract_frequency_domain_features(window, fs=1000):
    """Extract frequency-domain features from a signal window"""
    features = {}
    
    # FFT
    fft_vals = np.fft.fft(window)
    fft_freq = np.fft.fftfreq(len(window), 1/fs)
    fft_power = np.abs(fft_vals)**2
    
    # Only positive frequencies
    pos_mask = fft_freq > 0
    fft_freq_pos = fft_freq[pos_mask]
    fft_power_pos = fft_power[pos_mask]
    
    features['spectral_centroid'] = np.sum(fft_freq_pos * fft_power_pos) / (np.sum(fft_power_pos) + 1e-8)
    features['spectral_variance'] = np.sqrt(np.sum(((fft_freq_pos - features['spectral_centroid'])**2) * fft_power_pos) / (np.sum(fft_power_pos) + 1e-8))
    
    # Bearing fault frequencies (example for BPFI)
    bpfi_range = (fft_freq_pos >= 100) & (fft_freq_pos <= 200)
    features['bpfi_energy'] = np.sum(fft_power_pos[bpfi_range])
    
    return features

In [6]:
def example_test_single_file():
    """
    Example: Test model on a single pair of files
    """
    print("\n" + "="*70)
    print("EXAMPLE: Testing on Single File")
    print("="*70)
    
    # Specify your file paths
    temp_file = '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/0NM_current_temp_csv/0Nm_Normal.csv'  # Change to your file
    vib_file = '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/0nm_vibration_csv/0Nm_Normal.csv'      # Change to your file
    
    # Test the model
    predictions_df, y_pred, features = test_model_on_original_data(
        temp_filepath=temp_file,
        vib_filepath=vib_file,
        model_path = '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/CNN_LSTM model/predictive_maintenance_model.h5',
        scaler_X_path = '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/CNN_LSTM model/scaler_X.pkl',
        scaler_y_path = '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/CNN_LSTM model/scaler_y.pkl',
        sequence_length=10)
    
    # Visualize results
    if predictions_df is not None:
        visualize_test_predictions(predictions_df, y_pred)
    
    return predictions_df


def example_test_multiple_files():
    """
    Example: Test model on multiple files
    """
    print("\n" + "="*70)
    print("EXAMPLE: Testing on Multiple Files")
    print("="*70)


    temp_file = '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/0NM_current_temp_csv/'  # Change to your file
    vib_file = '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/0nm_vibration_csv/'      # Change to your file
    
    # Define all test files
    test_files = [
    ('/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/0NM_current_temp_csv/0Nm_Normal.csv',
     '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/0nm_vibration_csv/0Nm_Normal.csv',
     'Normal'),

    ('/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/0NM_current_temp_csv/0Nm_BPFI_03.csv',
     '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/0nm_vibration_csv/0Nm_BPFI_03.csv',
     'BPFI_03'),

    ('/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/0NM_current_temp_csv/0Nm_BPFI_10.csv',
     '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/0nm_vibration_csv/0Nm_BPFI_10.csv',
     'BPFI_10'),

    ('/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/0NM_current_temp_csv/0Nm_BPFI_30.csv',
     '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/0nm_vibration_csv/0Nm_BPFI_30.csv',
     'BPFI_30'),
                ]

    
    # Load model and scalers once
    model = '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/CNN_LSTM model/predictive_maintenance_model.h5'
    scaler_X = '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/CNN_LSTM model/scaler_X.pkl'
    scaler_y = '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/CNN_LSTM model/scaler_y.pkl'
    
    all_results = {}
    
    for temp_file, vib_file, condition in test_files:
        print(f"\n{'='*70}")
        print(f"Testing: {condition}")
        print(f"{'='*70}")
        
        try:
            predictions_df, y_pred, _ = test_model_on_original_data(
                temp_file, vib_file,
                model_path=model,
                scaler_X_path=scaler_X,
                scaler_y_path=scaler_y
            )
            
            all_results[condition] = {
                'predictions': predictions_df,
                'mean_rul': y_pred.mean(),
                'min_rul': y_pred.min(),
                'max_rul': y_pred.max()
            }
            
        except Exception as e:
            print(f"✗ Error testing {condition}: {e}")
    
    # Compare results across conditions
    print(f"\n{'='*70}")
    print("COMPARISON ACROSS CONDITIONS")
    print(f"{'='*70}")
    
    comparison_data = []
    for condition, results in all_results.items():
        comparison_data.append({
            'Condition': condition,
            'Mean RUL (min)': results['mean_rul'],
            'Min RUL (min)': results['min_rul'],
            'Max RUL (min)': results['max_rul']
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))
    comparison_df.to_csv('comparison_results.csv', index=False)
    print(f"\n✓ Comparison saved to 'comparison_results.csv'")
    
    return all_results


def example_real_time_monitoring():
    """
    Example: Simulate real-time monitoring
    """
    print("\n" + "="*70)
    print("EXAMPLE: Real-Time Monitoring Simulation")
    print("="*70)
    
    temp_file = '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/0NM_current_temp_csv/0Nm_Normal.csv'  # Change to your file
    vib_file = '/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/0nm_vibration_csv/0Nm_Normal.csv'      # Change to your file
    
    # Load and prepare data
    model = keras.models.load_model('/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/CNN_LSTM model/predictive_maintenance_model.h5',compile=False)
    scaler_X = joblib.load('/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/CNN_LSTM model/scaler_X.pkl')
    scaler_y = joblib.load('/media/harish/bc5d683c-2fdb-46e5-8916-190976536d13/Major project/Datasets/iteration 3/CNN_LSTM model/scaler_y.pkl')
    
    X_test_scaled, _ = load_and_prepare_test_data(temp_file, vib_file, scaler_X)
    
    # Simulate real-time predictions (every 10 sequences)
    print("\nSimulating real-time monitoring...")
    print("-" * 70)
    
    for i in range(0, len(X_test_scaled), 10):
        sequence = X_test_scaled[i:i+1]
        
        rul_scaled = model.predict(sequence, verbose=0)
        rul = scaler_y.inverse_transform(rul_scaled).flatten()[0]
        
        if rul > 60:
            status = "✓ HEALTHY"
            color = '\033[92m'  # Green
        elif rul > 20:
            status = "⚠ WARNING"
            color = '\033[93m'  # Yellow
        else:
            status = "✗ CRITICAL"
            color = '\033[91m'  # Red
        
        print(f"{color}Sequence {i:4d} | RUL: {rul:6.2f} min | {status}\033[0m")
    
    print("-" * 70)

In [7]:
example_real_time_monitoring()


EXAMPLE: Real-Time Monitoring Simulation

LOADING AND PREPARING TEST DATA

[1/5] Loading CSV files...


2025-10-27 00:27:56.301130: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


✓ Temperature data: (7682458, 6)
✓ Vibration data: (7680000, 5)

[2/5] Resampling from 25600Hz to 1000Hz...
✓ Temperature resampled: (300096, 6)
✓ Vibration resampled: (300000, 5)

[3/5] Extracting features from 2000 sample windows...
✓ Extracted features: (299, 103)
  Number of windows: 299
  Number of features per window: 103

[4/5] Creating sequences of length 10...
✓ Created 290 sequences
  Sequence shape: (290, 10, 103)

[5/5] Scaling features...
✓ Features scaled

DATA PREPARATION COMPLETE!
Ready for prediction: (290, 10, 103)

Simulating real-time monitoring...
----------------------------------------------------------------------
Sequence    0 | RUL:  88.03 min | ✓ HEALTHY
Sequence   10 | RUL:  80.85 min | ✓ HEALTHY
Sequence   20 | RUL:  88.65 min | ✓ HEALTHY
Sequence   30 | RUL:  82.58 min | ✓ HEALTHY
Sequence   40 | RUL:  86.97 min | ✓ HEALTHY
Sequence   50 | RUL:  81.49 min | ✓ HEALTHY
Sequence   60 | RUL:  89.07 min | ✓ HEALTHY
Sequence   70 | RUL:  79.71 min | ✓ HEALTHY
Se